In [1]:
import rasterio
import numpy as np

## Initialize

**training_regions** is an object containing image subregions for selecting pixels used to train and test the model. The keys correspond to different images (each of which contain 7 spectral bands). Whithin each image, three subregions were manually selected that represent some category we hope to identify with the model, such as "glacier" or "ocean".

**label_index** is an object that links category names, like "glacier", with the corresponding array index in the labels array. For now, there are only 3 categories, "glacier", "ocean", ["clouds", "nodata_land", "sea_ice", "land_snow"] => "other"

**band_postfix** are the file endings corresponding to different spectral bands in each image. "B1", "B2", etc. indicate the different bands

**features** are the variables calculated for each pixel. *x* and *y* are the geographic position of the pixel in projected coordinates. *I1* through *I7* are the pixel intensities in each of the seven spectal bands, B1 through B7

**samples_per_region** is the number of randomly selected pixels from each subregion.

In [2]:
training_regions = {'LE70322482009163EDC00': {'glacier': [504805, 9012826, 507745, 9015186], # [minx, miny, maxx, maxy]
                                              'ocean': [501577, 9033498, 505797, 9037183],
                                              'clouds': [459838, 8986574, 464556, 8990694]},
                    'LE70322482009195EDC00': {'glacier': [519943, 9014150, 520819, 9014914],
                                              'ocean': [494791, 9017577, 503515, 9025196],
                                              'nodata_land': [457395, 8959990, 474370, 8974814]},
                    'LE70332482010173EDC00': {'glacier': [510865, 9028302, 517171, 9033809],
                                              'ocean': [502533, 9038881, 509313, 9044802],
                                              'sea_ice': [468803, 9024503, 473521, 9028623]},
                    'LE70342482009177EDC00': {'glacier': [523709, 9011654, 524382, 9012241],
                                              'ocean': [478029, 9026676, 493486, 9040175],
                                              'land_snow': [529319, 9021811, 546625, 9036925]}}
label_index = {'glacier': 0, 'ocean': 1, 'clouds': 2, 'nodata_land': 2, 'sea_ice': 2, 'land_snow': 2}
band_postfix = ['_B1_clipped.TIF', '_B2_clipped.TIF', '_B3_clipped.TIF', '_B4_clipped.TIF',
                '_B5_clipped.TIF', '_B6_VCID_2_clipped.TIF', '_B7_clipped.TIF']
features = ['x', 'y', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7']
train_samples_per_region = 2000
test_samples_per_region = 100
num_images = len(training_regions)
num_categories = 3

### Construct training and testing sets

Iterate through each subregion in each image and construct the feature array (see *features* above) for each randomly selected pixel. The result is four tensors:

**training_set** shape(number of pixels, number of features)
```
           x    y   I1  I2  I3  I4  I5  I6  I7
pixel1
pixel2
pixel3
                        ...etc
```
**training_labels** shape(number of pixels, number of categories)
```
        glacier(0) ocean(1) other(2)
pixle1     0          0        1
pixel2     0          0        1
pixel3     1          0        0
                ...etc
```                
is a tensor containing "one-hot" vectors for each pixel. It has a "1" in the positioin of the know category for that pixel, and 0s elsewhere.

**test_set** is the same as training_set, but is based on a different random set of pixels and is used for testing the learned model

**test_labels** again, the same as the training_labels

In [4]:
training_set = np.array([]).reshape(0, len(features))
training_labels = np.array([]).reshape(0, num_categories)

test_set = np.array([]).reshape(0, len(features))
test_labels = np.array([]).reshape(0, num_categories)

for key, value in training_regions.items():
    for category, bounds in value.items():
        train_x = np.random.randint(bounds[0], bounds[2], train_samples_per_region)
        test_x = np.random.randint(bounds[0], bounds[2], test_samples_per_region)
        train_y = np.random.randint(bounds[1], bounds[3], train_samples_per_region)
        test_y = np.random.randint(bounds[1], bounds[3], test_samples_per_region)
        
        train_sub_set = np.zeros([train_samples_per_region, len(features)])
        train_sub_set[:,0] = train_x
        train_sub_set[:,1] = train_y

        test_sub_set = np.zeros([test_samples_per_region, len(features)])
        test_sub_set[:,0] = test_x
        test_sub_set[:,1] = test_y

        train_sub_labels = np.zeros([train_samples_per_region, num_categories])
        test_sub_labels = np.zeros([test_samples_per_region, num_categories])
        
        for feature_index, band in enumerate(band_postfix):
            path = './images/' + key + '/' + key + band
            with rasterio.open(path) as dataset:
                
                for index, [x, y] in enumerate(train_sub_set[:,0:2]):
                    [[I]] = list(dataset.sample([(x, y)]))
                    train_sub_set[index, 2 + feature_index ] = I
                    train_sub_labels[index, label_index[category]] = 1
                
                for index, [x, y] in enumerate(test_sub_set[:,0:2]):
                    [[I]] = list(dataset.sample([(x, y)]))
                    test_sub_set[index, 2 + feature_index] = I
                    test_sub_labels[index, label_index[category]] = 1
                     
        training_set = np.vstack([training_set, train_sub_set])
        training_labels = np.vstack([training_labels, train_sub_labels])
        test_set = np.vstack([test_set, test_sub_set])
        test_labels = np.vstack([test_labels, test_sub_labels])


### Save data

In [5]:
np.save('training_set', training_set)
np.save('training_labels', training_labels)
np.save('test_set', test_set)
np.save('test_labels', test_labels)